In [1]:
!pip install transformers
!pip install datasets
!pip install tensorflow

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00


In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
import tensorflow as tf

In [6]:
checkpoint = 'facebook/bart-large-xsum' # Model
tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [8]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [9]:
dataset = load_dataset("AjayMukundS/Legal_Text_Summarization-llama2")

train_test_split = dataset['train'].train_test_split(test_size=0.2)  # 80% train, 20% test

# Further split the test set into validation and test sets
test_valid_split = train_test_split['test'].train_test_split(test_size=0.5)  # 50% validation, 50% test

README.md:   0%|          | 0.00/494 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7773 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
from datasets import load_dataset, DatasetDict


split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

In [11]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["judgement"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# Assuming you have already created `split_dataset` containing train, validation, and test sets

# Preprocess the training dataset
tokenized_train = split_dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary']  # Adjust these columns as needed
)

# Preprocess the validation dataset
tokenized_val = split_dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary'] # Adjust these columns as needed
)

# Preprocess the test dataset
tokenized_test = split_dataset['test'].map(
    preprocess_function,
    batched=True,
    remove_columns=['judgement','dataset_name', 'text', 'summary']  # Adjust these columns as needed
)

# Print the structure of the tokenized datasets for verification
print("Tokenized Training Dataset:", tokenized_train)
print("Tokenized Validation Dataset:", tokenized_val)
print("Tokenized Test Dataset:", tokenized_test)


Map:   0%|          | 0/6218 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/777 [00:00<?, ? examples/s]

Map:   0%|          | 0/778 [00:00<?, ? examples/s]

Tokenized Training Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 6218
})
Tokenized Validation Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 777
})
Tokenized Test Dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 778
})


In [14]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset


training_args = Seq2SeqTrainingArguments(
    output_dir = 'bart_samsum',
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = 42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.859000,1.635790
2,1.298400,1.550516
3,1.173600,1.562785


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

TrainOutput(global_step=3108, training_loss=1.4338908692584535, metrics={'train_runtime': 3876.9337, 'train_samples_per_second': 6.415, 'train_steps_per_second': 0.802, 'total_flos': 5.38698861993001e+16, 'train_loss': 1.4338908692584535, 'epoch': 3.997427652733119})

In [22]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_resume_summarizer")

# Save the tokenizer
tokenizer.save_pretrained("./fine_tuned_resume_summarizer")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('./fine_tuned_resume_summarizer/tokenizer_config.json',
 './fine_tuned_resume_summarizer/special_tokens_map.json',
 './fine_tuned_resume_summarizer/vocab.json',
 './fine_tuned_resume_summarizer/merges.txt',
 './fine_tuned_resume_summarizer/added_tokens.json')

In [23]:
from transformers import pipeline, AutoTokenizer


summarizer = pipeline('summarization', model='/content/fine_tuned_resume_summarizer')


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [25]:
resume_text = """
EDWIN CHAZHOOR
Contact: 917069940011
GitHub: [Link] | LinkedIn: [Link]

EDUCATION
B.Tech in Computer Science Engineering (Specialization in AIML)
Vellore Institute of Technology, Vellore
Jul 2022 - Present
CGPA: 8.97

PROJECTS
- Personal Finance Tracker Web App: Developed a web application to help users manage their income, expenses, savings, and investments using Flask and MySQL.
- Smart Energy Management System (SEMS): Created a system that utilizes ensemble learning with AdaBoost and Random Forest classifiers to predict energy waste in smart homes.
- Hotel Management System: Designed and implemented a GUI-based application for efficient hotel management using Python and MySQL.
- AI-Powered Job Recommendation System: Developed a system that connects job seekers with relevant job opportunities by leveraging machine learning and natural language processing (NLP).
- Intrusion Detection System in IoT Devices: Automated threat detection and prevention using AI.

AWARDS & ACTIVITIES
- Held the TechnoVIT Ideathon for the ‘TrashCanGo’ app idea.
"""

In [26]:
# Prepare your resume text
#resume_text = "EDWIN CHAZHOOR 917069940011   Github  LinkedIn EDUCATION BTech Computer Science Engineering with specialization in AIML Vellore Institute of TechnologyChennai Jul 2022 Present CGPA 897 CBSE  XII Anand Vidya Vihar Vadodara Result 92 CBSE  X Anand Vidya Vihar Vadodara Result 86 PROJECTS May 2022 March 2020 Personal Finance Tracker Web App Feb 2024April 2024 Developed a Personal Finance Manager to help users organize and optimize their income expenses savings and investments Database Management Configured and optimized MySQL database to store and retrieve user data Utilized Flask to build a web application framework User Interface Design Created responsive and userfriendly web interfaces using HTML CSS and JS Github  Smart Energy Management System using Ensemble Learning Feb 2024May 2024 Developed an SEMS that minimizes energy wastage by combining the bestperforming ensemble learning classifier based on accuracy This project introduces a Smart Energy Management System SEMS that utilizes ensemble learning with AdaBoost and Random Forest classifiers to predict energy waste in smart homes with high accuracy Github Hotel Management System Jan 2022March 2022 Developed a Hotel Management System providing users with access to all essential features required for efficient hotel management such as room booking checkout and reservation Designed and developed the entire application using Python used Tkinter for building the graphical user interface GUI of the application used MySQL for managing and storing hotel data efficiently Github  AI powered job recommendation system System will connect job seekers with relevant job opportunities by leveraging machine learning and natural language processing NLP Intrusion Detection System in IOT devices using AI Automated threat detection and prevention SKILLS Technical Skills CC Python Java ReactJS JS HTMLCSS SQL Languages English Malayalam Hindi Spanish AwardsActivities Won the TechnoVIT Ideathon for ‘TrashCanGo’ app idea Ongoing Ongoing"

# Generate the summary
summary = summarizer(resume_text, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# Print the summary
print("Summary of the Resume:")
print(summary[0]['summary_text'])


Summary of the Resume:
Edwin Chazhoort is a computer science graduate from Vellore Institute of Technology in India.
He is working on a web application to help users manage their income, expenses, savings, and investments using Flask and MySQL.
The app was developed with the help of machine learning and natural language processing (NLP).
He also holds a B.Tech in Computer Science Engineering (Specialization in AIML).
His main project is a system that connects job seekers with relevant job opportunities using artificial intelligence (AI) powered job recommendation systems.
Edwin is also working on an application that predicts energy waste in smart homes.
In addition, he is developing a system to detect intrusions in IoT devices


In [27]:
!pip install transformers sentence-transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.7 MB/s eta 0:00:00


In [32]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the SBERT model (you can use Universal Sentence Encoder if preferred)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Or use 'all-MiniLM-L6-v2' for general matching

# Sample summarized resume and job description
summarized_resume =summary[0]['summary_text']
job_descriptions = [
    "Looking for a data scientist proficient in machine learning, deep learning, and data analysis.",
    "Hiring a software engineer with experience in cloud technologies like AWS.",
    "Seeking a project manager with strong skills in Agile methodologies and team leadership.",
    "We are seeking a skilled Data Scientist / Machine Learning Engineer to design and develop innovative AI-powered solutions that connect job seekers with relevant opportunities, manage financial applications, and enhance energy efficiency in smart homes. The ideal candidate will have a B.Tech in Computer Science Engineering with a specialization in AIML, proficiency in Python, Flask, and MySQL, and experience in machine learning and natural language processing. You will be responsible for developing and deploying predictive models, implementing intrusion detection systems for IoT devices, and collaborating with cross-functional teams to drive impactful projects that leverage cutting-edge technology. Join us to contribute to transformative solutions in the tech industry!"
]

resume_embedding = model.encode(summarized_resume)
job_embeddings = model.encode(job_descriptions)

resume_embedding = np.reshape(resume_embedding, (1, -1))
job_embedding = np.reshape(job_embeddings, (1, -1))

# Compute cosine similarities for each job description
similarity_scores = cosine_similarity(resume_embedding, job_embeddings)

# Print the similarity scores
for i, score in enumerate(similarity_scores[0]):
    print(f"Job {i+1} Similarity Score: {score*100}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Job 1 Similarity Score: 40.53361117839813
Job 2 Similarity Score: 46.23601734638214
Job 3 Similarity Score: 43.28780770301819
Job 4 Similarity Score: 58.79474878311157


In [33]:
best_match_idx = np.argmax(similarity_scores)
best_match_score = similarity_scores[0][best_match_idx]
best_match_description = job_descriptions[best_match_idx]

print(f"Best Matching Job Description: {best_match_description}")
print(f"Best Matching Similarity Score: {best_match_score}")


Best Matching Job Description: We are seeking a skilled Data Scientist / Machine Learning Engineer to design and develop innovative AI-powered solutions that connect job seekers with relevant opportunities, manage financial applications, and enhance energy efficiency in smart homes. The ideal candidate will have a B.Tech in Computer Science Engineering with a specialization in AIML, proficiency in Python, Flask, and MySQL, and experience in machine learning and natural language processing. You will be responsible for developing and deploying predictive models, implementing intrusion detection systems for IoT devices, and collaborating with cross-functional teams to drive impactful projects that leverage cutting-edge technology. Join us to contribute to transformative solutions in the tech industry!
Best Matching Similarity Score: 0.5879474878311157
